
#Structured Streaming

Demonstriramo simulaciju streaming-a koristecu podatke iz wikimedia i demonstriramo  ako bi se podaci konstatno menjali takodje bi i njeni rezultati.



In [0]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,LongType,BooleanType
from time import sleep
# Create a Spark session
spark = SparkSession.builder.appName("CSVStreamExample").getOrCreate()
# Read CSV as a streaming DataFrame
csv_path = "dbfs:/FileStore/wikimediaoutput"
schema1 = StructType([
    StructField("$schema", StringType(), True),
    StructField("id", LongType(), True),
    StructField("type", StringType(), True),
    StructField("namespace", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("title_url", StringType(), True),
    StructField("comment", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("user", StringType(), True),
    StructField("bot", BooleanType(), True),
    StructField("notify_url", StringType(), True),
    StructField("minor", BooleanType(), True),
    StructField("length", IntegerType(), True),
    StructField("revision", StringType(), True),
    StructField("server_url", StringType(), True),
    StructField("server_name", StringType(), True),
    StructField("server_script_path", StringType(), True),
    StructField("wiki", StringType(), True),
    StructField("parsedcomment", StringType(), True),
    StructField("length_new", IntegerType(), True),
    StructField("revision_new", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("request_id", StringType(), True),
    StructField("dt", StringType(), True),
    StructField("domain", StringType(), True),
    StructField("stream", StringType(), True),
    StructField("topic", StringType(), True),
    StructField("partition", IntegerType(), True),
    StructField("offset", LongType(), True),
])
from pyspark.sql.functions import *
%matplotlib inline
streaming_df=spark.readStream.schema(schema1).option("maxFilesPerTrigger", 1).csv(csv_path)


#query.awaitTermination()


In [0]:
streamingCountsDF=(streaming_df.groupBy("bot").count())
#display(streamingCountsDF)
print(streamingCountsDF.isStreaming)


query = (
    streamingCountsDF
    .writeStream
    .outputMode("complete")
    .format("memory")  # Use 'memory' to store in-memory table
    .queryName("counts")
   .trigger(processingTime="1 seconds")
    .start()
)

True


In [0]:
while not query.recentProgress:
    sleep(1)


Ovo je jedan takav primer gde se menjaju podaci, grupisali smo podatke vezane za "bot" i sabrali smo ih

In [0]:
%sql select * from counts order by count desc;

bot,count
false,2241
null,2165
true,1227


In [0]:
sleep(5)

In [0]:
%sql select * from counts  order by count desc;

bot,count
false,5209
null,4383
true,3107


In [0]:
sleep(5)

In [0]:
%sql select * from counts order by count desc;

bot,count
false,7257
null,6425
true,4389


In [0]:
query.stop()